In [22]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [39]:

train_D = pd.read_csv("Davis_train_origin.csv", header = 0)
test_D = pd.read_csv("Davis_test_origin.csv", header = 0)
val_D = pd.read_csv("Davis_val_origin.csv", header = 0)

train_D.drop("Unnamed: 0", axis = 1, inplace = True)
test_D.drop("Unnamed: 0", axis = 1, inplace = True)
val_D.drop("Unnamed: 0", axis = 1, inplace = True)

unique_drug = train_D['Drug_ID'].value_counts().index + test_D['Drug_ID'].value_counts().index + val_D['Drug_ID'].value_counts().index

for i in unique_drug :
    print(i)
    
print(len(unique_drug))

36592664
47234832
40212994
5657004
16947982
36927988
20503770
14704211
10413764
25970421
25212989
11001354
46914109
29490539
16701436
54131391
57857405
21945640
5611919
70818163
36839264
60821081
29933475
25548452
8358214
31604021
62999484
56583743
21992907
17035037
46261704
10114921
23329716
3391774
31149573
20593797
58503486
52529653
11698135
21729518
31558798
58575674
12510889
13366808
33257016
34857203
19942943
10258631
30744538
27242820
44422319
26337608
61949932
50833236
28050539
10662864
24320643
79120497
28070182
32837026
13632704
318127
30218887
37896759
34839764
58369882
28612018
28142750
68


In [40]:
protein2gene = pd.read_csv("convert_table", delimiter="\t")

In [42]:
pubchem2drugbank =  pd.read_csv("mbrole2_conversion06232318.tsv", delimiter='\t')

#Select the dataset that you want to make as (entity1, entity2, rel)
# options val_D, test_D, train_D
origin_dti =  train_D[train_D['Drug_ID'].isin(np.array(pubchem2drugbank['Input']))][['Drug_ID', "Target_ID", "Y"]]

def pub_to_db(x) : 
    return pubchem2drugbank[pubchem2drugbank['Input'] == x]['Output'].values[0]

def pro_to_gene(x) :
    
    result = protein2gene[protein2gene['Gene name'] == x]['NCBI gene (formerly Entrezgene) ID'].values
    
    if len(result) >= 1 :
        return result[0]
    else :
        return 1

dti = pd.DataFrame()

dti['Drug'] = origin_dti['Drug_ID'].apply(pub_to_db)
dti['Target'] = origin_dti['Target_ID'].apply(pro_to_gene)

In [43]:
dti['rel'] = origin_dti["Y"]

In [44]:
dti = dti[dti['Target'] != 1]

In [45]:
dti

,Drug,Target,rel
0,DB08568,Gene::22848,0
2,DB08568,Gene::27,0
3,DB08568,Gene::90,0
4,DB08568,Gene::92,0
7,DB08568,Gene::207,0
...,...,...,...
18028,DB04879,Gene::25989,0
18030,DB04879,Gene::7444,0
18031,DB04879,Gene::7465,0
18032,DB04879,Gene::494551,0


In [46]:
import pandas as pd
import numpy as np
import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T

In [47]:
dti

,Drug,Target,rel
0,DB08568,Gene::22848,0
2,DB08568,Gene::27,0
3,DB08568,Gene::90,0
4,DB08568,Gene::92,0
7,DB08568,Gene::207,0
...,...,...,...
18028,DB04879,Gene::25989,0
18030,DB04879,Gene::7444,0
18031,DB04879,Gene::7465,0
18032,DB04879,Gene::494551,0


In [31]:
#a=a.drop("gene_inco", axis = 1)
temp_list = []
for i in dti['Target'].copy() :
    exist = False
    for cn, j in enumerate(df.index) :
        length = len(df.index)
        if i == j :
            exist = True
            val = int(df[0].loc[j])
            temp_list.append(val)
            break
        elif cn == length -1 :
            temp_list.append(np.nan)
            

In [32]:
dti['gene_enco'] = temp_list

In [33]:
dti

,Drug,Target,rel,gene_enco
2,DB06589,Gene::8737,0,3179.0
4,DB08142,Gene::9212,0,3113.0
6,DB00619,Gene::3932,0,2439.0
7,DB01259,Gene::4214,0,2032.0
11,DB02567,Gene::8569,0,3913.0
...,...,...,...,...
2567,DB04879,Gene::2044,0,7136.0
2569,DB01259,Gene::207,0,2232.0
2570,DB00619,Gene::4485,0,13250.0
2571,DB06616,Gene::4751,0,5991.0


In [34]:
import json

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)
    
    
#data   

In [35]:
temp_list2 = [] 
for i in dti["Drug"].copy() :
    exist = False
    count = 0
    for name, number in data.items() :
        length = len(data)
        if i == name :
            exist = True
            temp_list2.append(number)
            break
        elif count >= length -1 :
            temp_list2.append(np.nan)
        count += 1
len(temp_list2)

670

In [36]:
dti['drug_enco'] = temp_list2
dti = dti.dropna()
dti['gene_enco'] = dti['gene_enco'].astype(int)
dti['drug_enco'] = dti['drug_enco'].astype(int)


/tmp/ipykernel_1406445/972249599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['gene_enco'] = dti['gene_enco'].astype(int)
/tmp/ipykernel_1406445/972249599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['drug_enco'] = dti['drug_enco'].astype(int)


In [37]:
dti

,Drug,Target,rel,gene_enco,drug_enco
2,DB06589,Gene::8737,0,3179,437
6,DB00619,Gene::3932,0,2439,637
7,DB01259,Gene::4214,0,2032,206
21,DB08916,Gene::1859,0,4383,1549
27,DB01254,Gene::388228,0,5843,764
...,...,...,...,...,...
2558,DB04868,Gene::10746,0,5548,1115
2559,DB00619,Gene::4216,0,3491,637
2569,DB01259,Gene::207,0,2232,206
2570,DB00619,Gene::4485,0,13250,637


In [38]:
dti[['gene_enco', 'drug_enco', 'rel']].to_csv('dev.txt', sep=' ', index=False)

# you have just created one of three datasets if you want to make triplets, repeat the code with changing variable above 